In [15]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

#### Technology data

In [16]:
table_path = '/home/bmurmann/eda/designs/Book-on-gm-ID-design/starter_files_open_source_tools/ihp-sg13g2/simulation/'
n = lk(table_path+'sg13_lv_nmos.mat')

#### Specifications

In [17]:
av0 = 4; rs = 10e3; cl = 200e-15

#### Design choices

In [18]:
l = 0.13; wfing = 5; gm_id = 15; rd = 1e3


#### Sizing and benchmarking

In [19]:
gmrds = n.lookup('GM_GDS', GM_ID=gm_id, VDS=0.4, L=l)
gm = 1/rd*(1/av0 - 1/gmrds)**-1
id = gm/gm_id 
w = id/n.lookup('ID_W', GM_ID=gm_id, L=l)
nf = 1+np.floor_divide(w, wfing)

# Compute capacitances
cgs = w*n.lookup('CGS_W', GM_ID=gm_id, L=l)
cgd = w*n.lookup('CGD_W', GM_ID=gm_id, L=l)
cdd = w*n.lookup('CDD_W', GM_ID=gm_id, L=l)
cdb = cdd-cgd
cltot = cl+cdb

# Estimate bandwidth
rds = gmrds/gm
rdtot = rds*rd/(rds+rd)
tau1 = rs*(cgs + cgd*(1+av0)) + rdtot*(cltot+cgd)
bw = 1/2/np.pi/tau1

In [20]:
df = pd.DataFrame([id, w, nf, gm, gm_id, 1/rds, cgs, cgd, cdb, bw], ['id', 'w', 'nf', 'gm', 'gm_id', 'gds', 'cgs', 'cgd', 'cdb', 'bw'], columns=['value']); df

,value
id,3.759662e-04
w,6.228757e+01
nf,1.300000e+01
gm,5.639492e-03
gm_id,1.500000e+01
gds,4.098731e-04
cgs,5.181324e-14
cgd,3.723875e-14
cdb,2.042552e-14
bw,6.210132e+07


In [21]:
with open('example2_params.spice', 'w') as file:
    file.write(".param id = " + "{:.2e}".format(id) + '\n')
    file.write(".param w = " + "{:.2e}".format(w*1e-6) + '\n')
    file.write(".param l = " + "{:.2e}".format(l*1e-6) + '\n')
    file.write(".param nf = " + "{}".format(nf) + '\n')
    file.write(".param rd = " + "{:.2e}".format(rd) + '\n')
    file.write(".param rs = " + "{:.2e}".format(rs) + '\n')
    file.write(".param cl = " + "{:.2e}".format(cl) + '\n')